In [2]:
import time
import random
import os

import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import DebertaTokenizer, TFDebertaForSequenceClassification
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# These auto classes load the right type of tokenizer and model based on a model name
from transformers import AutoTokenizer, TFAutoModel

In [3]:
# set seed, TF uses python ramdom and numpy library, so these must also be fixed
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['PYTHONHASHSEED']=str(0)
os.environ['TF_DETERMINISTIC_OPS'] = '0'

In [26]:
def read_media_cloud_data(path, label):
    """Read in data downloaded from media cloud and assign a label to all rows"""
    df = pd.read_csv(path)
    df['Label_bias'] = label
    df = df.rename({'title': 'sentence'}, axis=1)
    return df

# read in two datasets
PATH_biased = "data/news_headlines_usa_biased.csv"
PATH_neutral = "data/news_headlines_usa_neutral.csv"
df_biased = read_media_cloud_data(PATH_biased, 1)
df_neutral = read_media_cloud_data(PATH_neutral, 0)

# combine them
df_distant = pd.concat([df_biased,df_neutral], axis=0, ignore_index=1)
df_distant = shuffle(df_distant)

# train-test split
df_distant_train, df_distant_test = train_test_split(df_distant, test_size=0.2)

# test pipeline set
df_distant, exclude = train_test_split(df_distant, test_size=0.95)
df_distant_train, df_distant_test = train_test_split(df_distant, test_size=0.2)

In [27]:
def preprocess(df, model_name):
    """convert a pandas dataframe into a tensorflow dataset"""
    df2 = df.copy(deep=False)
    target = df2.pop('Label_bias')
    sentence = df2.pop('sentence')

    if model_name=='bert':
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    elif model_name=='roberta':
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    elif model_name=='deberta':
        tokenizer = DebertaTokenizer.from_pretrained("kamalkraj/deberta-base")

    train_encodings = tokenizer(
                        sentence.tolist(),                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        truncation = True, # cut off at max length of the text that can go to BERT
                        padding = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
    
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(train_encodings), 
         target.tolist()))
    return dataset

In [29]:
def train_model(train_df, test_df, model_name):
    # pandas -> tensorflow
    train_distant_dataset = preprocess(train_df, model_name)
    test_distant_dataset = preprocess(test_df, model_name)

    # batch and randomize
    BUFFER_SIZE = 10000
    BATCH_SIZE = 24

    train_distant_dataset = train_distant_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    test_distant_dataset = test_distant_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    
    tf.keras.backend.clear_session()
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True) # after 3 epochs without improvement, stop training
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    
    if model_name=='bert':
        clf = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
    elif model_name=='roberta':
        clf = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
    elif model_name == 'deberta':
        clf = TFDebertaForSequenceClassification.from_pretrained("kamalkraj/deberta-base")

    clf.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy') 
    history = clf.fit(train_distant_dataset, epochs=1, validation_data = test_distant_dataset, callbacks=[callback])
    trained_layer = clf.get_layer(index=0).get_weights()
    clf.save_weights(f'./checkpoints/{model_name}_final_checkpoint_news_headlines_USA')

In [30]:
models = ['bert', 'roberta', 'deberta']

for model in models:
    train_model(df_distant_train, df_distant_test, model)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


215/215 [==============================] - 75s 276ms/step - loss: 3.0046 - accuracy: 0.3488 - val_loss: 2.9835 - val_accuracy: 0.3533


2022-10-28 16:10:57.838858: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154414080 exceeds 10% of free system memory.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


215/215 [==============================] - ETA: 0s - loss: 0.6397 - accuracy: 0.4737

2022-10-28 16:12:18.057290: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154414080 exceeds 10% of free system memory.


...

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Downloading:   0%|          | 0.00/555M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at kamalkraj/deberta-base and are newly initialized: ['pooler', 'classifier', 'cls_dropout']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
215/215 [==============================] - 153s 567ms/step - loss: 6.9446 - accuracy: 0.6461 - val_loss: 7.6550 - val_accuracy: 0.6545
